In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:
from src.models.encoder import Encoder
import torch 

In [76]:
encoder = Encoder('resnet18')

In [78]:
channels = [
	encoder.encoder.feature_info.channels(i)
	for i in range(len(encoder.encoder.feature_info))
]

channels

[64, 64, 128, 256, 512]

In [73]:
features = encoder(torch.rand(1, 3, 224, 224))

In [74]:
for f in features:
	print(f.shape)

torch.Size([1, 64, 112, 112])
torch.Size([1, 64, 56, 56])
torch.Size([1, 128, 28, 28])
torch.Size([1, 256, 14, 14])
torch.Size([1, 512, 7, 7])


In [53]:
# up

x = features[-1]
up = [x]
for i in reversed(range(1, len(features))):
	# print(x.shape)
	xr = torch.nn.functional.interpolate(x, scale_factor=2, mode='bilinear')
	# print('reshaped', xr.shape)
	# print('previous', features[i-1].shape)
	x = torch.cat((features[i-1], xr), 1)
	# print('concat', x.shape)
	up.append(x)
	# x = torch.nn.Conv2d(x.shape[1], features[i-1].shape[1], 3, padding=1)(x)
	# print('conv', x.shape)

In [54]:
for u in up:
	print(u.shape)

torch.Size([1, 512, 7, 7])
torch.Size([1, 768, 14, 14])
torch.Size([1, 896, 28, 28])
torch.Size([1, 960, 56, 56])
torch.Size([1, 1024, 112, 112])


In [70]:
# down 

down = [up[-1]]
for i in reversed(range(1, len(up))):
	xc = torch.nn.Conv2d(up[i].shape[1], up[i-1].shape[1], 3, padding=1)(up[i])
	xcm = torch.nn.MaxPool2d(2)(xc)
	# print('conv', xc.shape)
	xcc = torch.cat((xcm, up[i-1]), 1)
	down.append(xcc)


In [71]:
for d in down:
	print(d.shape)

torch.Size([1, 1024, 112, 112])
torch.Size([1, 1920, 56, 56])
torch.Size([1, 1792, 28, 28])
torch.Size([1, 1536, 14, 14])
torch.Size([1, 1024, 7, 7])


In [86]:
accum = channels[-1:]
for i in reversed(range(len(channels)-1)):
	accum.append(accum[-1] + channels[i])
accum

[512, 768, 896, 960, 1024]

In [107]:
down_channels = accum[-1:]
for i in range(len(accum)-1):
	down_channels.append(accum[::-1][i+1]*2)
down_channels

[1024, 1920, 1792, 1536, 1024]